In [1]:
# Problema da mochila 0-1
import sys
import numpy as np
import gurobipy as gb
from gurobipy import GRB

In [3]:
# algoritmo de dantzing

def lp (n, p, w, c, x):

    v = np.divide(p, w) # p/w
    #print("v = ", np.round(v,2))
    
    # indices de v em ordem não-decrescente
    idx = np.argsort(v, kind='quicksort')[::-1]
    #print("idx = ", idx)
    
    # faz cópia da capacidade
    aux = c 
    
    for i in idx:
        if aux >= w[i]: # verifica se existe espaço na mochila
            aux = aux - w[i] # atualiza a capacidade da mochila
            x[i] = 1 # atualiza a solução
        else:
            xh = aux/w[i] # calculo da proporção que cabe na mochila
            x[i] = xh # atualiza a solução
            break
            
    # valor ótimo
    opt = sum(p*x)
    
    return opt 

In [4]:
# mip

def mip (n, p, w, c, xmip):
    
    # cria modelo
    model = gb.Model()
        
    # define variáveis
    x = model.addVars(n, vtype=GRB.BINARY, name='x')
    
    obj = None
    for i in range(0, n):
        obj += p[i] * x[i]
 
    model.setObjective(obj, GRB.MAXIMIZE)
    
    # desabilita print do solver
    model.Params.LogToConsole = 0 

    constr = None
    for j in range(0, n):
        constr += (w[j] * x[j])
    model.addConstr(constr <= c)
        
    # resolve o problema
    model.optimize()
    
    for v in model.getVars():
        xmip.append(v.x)
        
    return model.objVal

In [5]:
# cover

def cover (n, p, w, c, xc):
    
    # cria modelo
    model = gb.Model()
       
    # define variáveis
    x = model.addVars(n, vtype=GRB.BINARY, name='x')
    
    obj = None
    for i in range(0, n):
        obj += p[i] * x[i]
 
    model.setObjective(obj, GRB.MINIMIZE)
    
    # desabilita print do solver
    model.Params.LogToConsole = 0 

    constr = None
    for j in range(0, n):
        constr += (w[j] * x[j])
    model.addConstr(constr >= c+1)
        
    # resolve o problema
    model.optimize()
    
    # pegando solução
    for v in model.getVars():
        xc.append(v.x)
        
    return model.objVal

In [7]:
if __name__ == "__main__":
    
    instancia = 2
    
    if instancia == 1:
        p = [6, 5, 8, 9, 6, 7, 3] # vetor de lucros
        w = [2, 3, 6, 7, 5, 9, 3] # vetor de pesos
        c = 9 # capacidade
        n = len(p) # número de itens
    else:
        
        datafile = "instances/kp/100/100_100_3.txt"
        
        with open(datafile, 'r') as file: 
            linhas = file.readlines()
        
        linhas = [a.strip() for a in linhas] # remove linha vazia inicial e elimina os "\n" de cada linha
        
        n = int(linhas[0]) # ler o tamanho da instancia
        
        p = np.fromstring(linhas[1], dtype=int, sep = ' ') # ler a diagonal da matriz
                    
        c = int(linhas[2]) # ler a capacidade
        
        w = np.fromstring(linhas[3], dtype=int, sep = ' ') # ler os pesos
    
    
    xlp = np.zeros(n)
    xmip = []    
    z =[]
        
    # resolve relaxação linear
    print("Relaxação linear")
    opt = lp (n, p, w, c, xlp)
    
    print("x = [", end = " ")
    for i in range(n):
        print(xlp[i], end = ", ")
    print("]")
    
    print("ótimo = ", np.round(opt,2))
    print("\n")
    
    N = range(len(xlp))
    
    # lucros a serem usados no problema de separação de cover
    newp = np.zeros((n))
    for i in N:
        newp[i] = 1 - xlp[i]
        
    # encontra cover
    print("Encontrando desigualdade de cobertura")
    opt = cover (n, newp, w, c, z)

    print("x = [", end = " ")
    for i in range(n):
        print(z[i], end = ", ")
    print("]")


    print("ótimo = ", np.round(opt,2))
    print("\n")

    delta = 1 - opt

    if delta < 1:
        print("Delta %g" % delta)
        print("Desigualdade de cobertura violada pelo lp")
        right = 0
        for i in N:
            if z[i]:
                if (i > 0):
                    print('+',end =" ")
                print('x[%s]' % i, end =" ")
                right += 1
        print(" <= ", end=" ")
        print(right-1)

Relaxação linear
x = [ 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.75, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ]
ótimo =  2385.5


Encontrando desigualdade de cobertura
x = [ 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, -0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, -0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 